In [27]:
import pandas as pd  # Import pandas for data manipulation and analysis
import numpy as np  # Import numpy for numerical operations
from sklearn.model_selection import train_test_split  # Import train_test_split for splitting data into training and testing sets
from sklearn.preprocessing import StandardScaler, LabelEncoder  # Import StandardScaler for feature scaling and LabelEncoder for encoding target labels
from xgboost import XGBClassifier  # Import XGBClassifier for XGBoost model
from lightgbm import LGBMClassifier  # Import LGBMClassifier for LightGBM model
from catboost import CatBoostClassifier  # Import CatBoostClassifier for CatBoost model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score  # Import specific metrics for model evaluation
import warnings  # Import warnings to manage warnings
import category_encoders as ce  # Import category_encoders for encoding categorical features
import optuna
# Ignore warnings
warnings.filterwarnings('ignore')  # Suppress warnings for cleaner output

In [28]:
path = r'C:\Users\User\Desktop\Rashad\DATA\Thyroid_Diff.csv'

# Load dataset from Excel file
df = pd.read_csv(path)

# Set option to display all columns
pd.set_option('display.max_columns', None)

#Display the loaded dataset
df

,Age,Gender,Smoking,Hx Smoking,Hx Radiothreapy,Thyroid Function,Physical Examination,Adenopathy,Pathology,Focality,Risk,T,N,M,Stage,Response,Recurred
0,27,F,No,No,No,Euthyroid,Single nodular goiter-left,No,Micropapillary,Uni-Focal,Low,T1a,N0,M0,I,Indeterminate,No
1,34,F,No,Yes,No,Euthyroid,Multinodular goiter,No,Micropapillary,Uni-Focal,Low,T1a,N0,M0,I,Excellent,No
2,30,F,No,No,No,Euthyroid,Single nodular goiter-right,No,Micropapillary,Uni-Focal,Low,T1a,N0,M0,I,Excellent,No
3,62,F,No,No,No,Euthyroid,Single nodular goiter-right,No,Micropapillary,Uni-Focal,Low,T1a,N0,M0,I,Excellent,No
4,62,F,No,No,No,Euthyroid,Multinodular goiter,No,Micropapillary,Multi-Focal,Low,T1a,N0,M0,I,Excellent,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,72,M,Yes,Yes,Yes,Euthyroid,Single nodular goiter-right,Right,Papillary,Uni-Focal,High,T4b,N1b,M1,IVB,Biochemical Incomplete,Yes
379,81,M,Yes,No,Yes,Euthyroid,Multinodular goiter,Extensive,Papillary,Multi-Focal,High,T4b,N1b,M1,IVB,Structural Incomplete,Yes
380,72,M,Yes,Yes,No,Euthyroid,Multinodular goiter,Bilateral,Papillary,Multi-Focal,High,T4b,N1b,M1,IVB,Structural Incomplete,Yes
381,61,M,Yes,Yes,Yes,Clinical Hyperthyroidism,Multinodular goiter,Extensive,Hurthel cell,Multi-Focal,High,T4b,N1b,M0,IVA,Structural Incomplete,Yes


In [29]:
df['Recurred'] = df['Recurred'].map({'No': 0, 'Yes': 1})

In [31]:
models = ["xgboost", "lightgbm", "catboost", "catboost_custom"]

# Create copies of the DataFrame for different preprocessing approaches
df_boost = df.copy()
df_cat = df.copy()

# Iterate through each model in the list
for model in models:            
    if model in ["xgboost", "lightgbm", "catboost"]:
        # Preprocessing for tree-based models
        
        # Perform Weight of Evidence Encoding on all columns except the target column 'default'
        all_columns = df_boost.columns.tolist()
        all_columns.remove('Recurred')
        encoder = ce.WOEEncoder(cols=all_columns)
        df_boost[all_columns] = encoder.fit_transform(df_boost[all_columns], df_boost['Recurred'])
             
    elif model ==  "catboost_custom":
        # Preprocessing for CatBoost with custom handling of missing values
        
        # Fill missing values in specified categorical columns with 'Missing Value'
        numerical_cols = df_cat.select_dtypes(include=['int64', 'float64']).columns.tolist()
        numerical_cols.remove('Recurred')
        columns_to_fill = cat_features=['Gender', 'Smoking', 'Hx Smoking', 'Hx Radiothreapy',
       'Thyroid Function', 'Physical Examination', 'Adenopathy', 'Pathology',
       'Focality', 'Risk', 'T', 'N', 'M', 'Stage', 'Response']
        df_cat[columns_to_fill] = df_cat[columns_to_fill].fillna('Missing Value')
        
        # Perform Weight of Evidence Encoding on numerical columns
        cat_encoder = ce.WOEEncoder(cols=numerical_cols)
        df_cat[numerical_cols] = cat_encoder.fit_transform(df_cat[numerical_cols], df_cat['Recurred'])

# Split data into features (X) and target (y) for each preprocessing approach
x_boost = df_boost.drop(columns=['Recurred'])
y_boost = df_boost['Recurred']
X_train_boost, X_test_boost, y_train_boost, y_test_boost = train_test_split(x_boost, y_boost, test_size=0.3, random_state=42)

x_cat = df_cat.drop(columns=['Recurred'])
y_cat = df_cat['Recurred']
X_train_cat, X_test_cat, y_train_cat, y_test_cat = train_test_split(x_cat, y_cat, test_size=0.3, random_state=42)

In [32]:
df.select_dtypes(include='object').columns

Index(['Gender', 'Smoking', 'Hx Smoking', 'Hx Radiothreapy',
       'Thyroid Function', 'Physical Examination', 'Adenopathy', 'Pathology',
       'Focality', 'Risk', 'T', 'N', 'M', 'Stage', 'Response'],
      dtype='object')

In [33]:
models = []

xgb_model_def = XGBClassifier()
lgb_model_def = LGBMClassifier()
catboost_model_def = CatBoostClassifier()
catboost_model_custom = CatBoostClassifier(cat_features=['Gender', 'Smoking', 'Hx Smoking', 'Hx Radiothreapy',
                   'Thyroid Function', 'Physical Examination', 'Adenopathy', 'Pathology',
                   'Focality', 'Risk', 'T', 'N', 'M', 'Stage', 'Response'])

models.extend([
    ('XGBoost', xgb_model_def),
    ('LightGBM', lgb_model_def),
    ('CatBoost', catboost_model_def),
    ('CatBoost_Custom', catboost_model_custom),
])

In [34]:
def train_and_evaluate_model(model_name, model, X_train, y_train, X_test, y_test):
    """
    Train and evaluate the given model on the training and testing data.

    Parameters:
    model_name (str): Name of the model for display purposes.
    model : Machine learning model object.
    X_train : Features of the training data.
    y_train : Target labels of the training data.
    X_test : Features of the testing data.
    y_test : Target labels of the testing data.

    Returns:
    float: Gini coefficient calculated from the model's predictions.
    """

    # Fit the model on the training data
    model.fit(X_train, y_train)

    # Predict labels and probabilities on the testing data
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    # Calculate ROC AUC and Gini coefficient
    roc_prob = roc_auc_score(y_test, y_prob)
    gini_prob = roc_prob * 2 - 1

    # Calculate confusion matrix and classification report
    confusion_matrix_result = confusion_matrix(y_test, y_pred)
    classification_report_result = classification_report(y_test, y_pred)

    # Print model performance metrics
    print(f'Model Performance for {model_name}')
    print('Gini prob is', gini_prob * 100)
    print(classification_report_result)
    print(confusion_matrix_result)

    return gini_prob

In [35]:
gini_df = pd.DataFrame(columns=['Model', 'Gini_prob'])

# Iterate through each model in the list of models
for model_name, model in models:
    # Train and evaluate the model, and calculate the Gini coefficient
    if model_name == 'CatBoost_Custom':
        gini_prob = train_and_evaluate_model(model_name, model, X_train_cat, y_train_cat, X_test_cat, y_test_cat)
    else:
        gini_prob = train_and_evaluate_model(model_name, model, X_train_boost, y_train_boost, X_test_boost, y_test_boost)
        
    # Add model name and Gini coefficient to the DataFrame
    if gini_prob is not None:
        gini_df = pd.concat([gini_df, pd.DataFrame({'Model': [model_name], 'Gini_prob': [gini_prob]})], ignore_index=True)

# Sort the DataFrame by Gini coefficient in descending order
gini_df_sorted = gini_df.sort_values(by='Gini_prob', ascending=False)

Model Performance for XGBoost
Gini prob is 97.81626506024097
              precision    recall  f1-score   support

           0       0.98      0.96      0.97        83
           1       0.91      0.94      0.92        32

    accuracy                           0.96       115
   macro avg       0.94      0.95      0.95       115
weighted avg       0.96      0.96      0.96       115

[[80  3]
 [ 2 30]]
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 76, number of negative: 192
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001152 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 85
[LightGBM] [Info] Number of data points in the train set: 268, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.283582 -> initscore=-0.926762
[LightGBM] [Info] 

In [36]:
gini_df_sorted

,Model,Gini_prob
2,CatBoost,0.987952
3,CatBoost_Custom,0.985693
0,XGBoost,0.978163
1,LightGBM,0.976657
